In [1]:
import pandas as pd
import numpy as np

In [2]:
# import list of incidents
raw_data = pd.read_csv("~/2019-datavis-final-project/data-csv/AllComplaintsClean.csv")
raw_data.columns

Index(['id', 'dispatchComplaint', 'Providerprimaryimpression', 'diagnosis'], dtype='object')

In [3]:
# keep only the complaint, impression, & diagnosis columns
raw_data = pd.DataFrame(data={"complaint":raw_data.dispatchComplaint, 
                              "impression":raw_data.Providerprimaryimpression, 
                              "diagnosis":raw_data.diagnosis})
# make a new column called "path" by concatenating the others
raw_data["path"] = raw_data["complaint"]+":"+raw_data["impression"]+":"+raw_data["diagnosis"]
# calculate frequencies of all paths in the data
raw_path_frequencies = raw_data.path.value_counts()
# add these frequencies in as a new column
raw_data["path_freq"] = raw_data["path"].apply(lambda x: raw_path_frequencies[x])
raw_data.head()

complaint                           impression  \
0    Abdominal Pain Problems                                 Pain   
1    Abdominal Pain Problems                       Abdominal Pain   
2  Hemorrhage Laceration ion          Gastrointestinal Hemorrhage   
3  Hemorrhage Laceration ion                Gi Bleed Bloody Stool   
4  Hemorrhage Laceration ion  Hemorrhage Not Elsewhere Classified   

                                     diagnosis  \
0  Postprocedural Hematoma Of A Dgstv Sys Org    
1    Postprocedural Seroma Of A Dgstv Sys Org    
2     Postprocedural Hemor Of A Dgstv Sys Org    
3     Postprocedural Hemor Of A Dgstv Sys Org    
4     Postprocedural Hemor Of A Dgstv Sys Org    

                                                path  path_freq  
0  Abdominal Pain Problems:Pain:Postprocedural He...          1  
1  Abdominal Pain Problems:Abdominal Pain:Postpro...          1  
2  Hemorrhage Laceration ion:Gastrointestinal Hem...          1  
3  Hemorrhage Laceration ion:Gi Bleed Bloody Stoo...          1  
4  Hemorrhage Laceration ion:Hemorrhage Not Elsew...          1

In [4]:
raw_data.describe()

path_freq
count  40131.000000
mean      17.455309
std       43.693023
min        1.000000
25%        1.000000
50%        2.000000
75%        9.000000
max      272.000000

In [5]:
def freq_stats(series):
    # for looking at the differences between various subsets of the data
    print("   count:", series.count())
    print("   sum:", series.sum())
    print("   mean:", series.mean())
    print("   std:", series.std())
    quantiles = series.quantile(q=[0,0.25,0.5,0.75,1])
    iqr = quantiles[0.75]-quantiles[0.25]
    print("   iqr:", iqr)
    print("   quantiles:", end="")
    quantiles.apply(lambda x: print("[ %d ]" %x, end=""))
    print("\n")

In [6]:
def get_top_five(series):
    # takes a pandas series argument
    # gets list of unique values & their frequencies
    # returns the top 5 most common values & their frequencies
    frequencies = series.value_counts()
    #print("get top5:",frequencies.count())
    return frequencies[0:5]

In [7]:
def get_top_paths(dataframe):
    # get top5 complaints,
    #     top5 impressions for each of those complaints,
    #   & top5 diagnoses for each of those impressions
    # generate a table of the resulting 125 (5^3) paths & their frequencies
    top125_paths = pd.DataFrame(columns=["complaint","impression","diagnosis","path_freq"])

    # Here's the plan:
    #     start w/ entire dataframe
    #     get top5 list of complaints
    #     for each complaint:
    #         subset dataframe
    #         get top5 list of impressions
    #         for each impression:
    #             subset the subsetted dataframe
    #             get top5 list of diagnoses
    #             for each diagnosis & frequency:
    #                 add to the list: [complaint, impression, diagnosis, frequency]
    top5_complaints = get_top_five(dataframe.complaint)
    for complaint in top5_complaints.index:
        #print("Complaint:", complaint)
        complaint_subset = dataframe.loc[dataframe.complaint == complaint]
        top5_impressions = get_top_five(complaint_subset.impression)
        for impression in top5_impressions.index:
            #print("   Impression:", impression)
            impression_subset = complaint_subset.loc[complaint_subset.impression == impression]
            top5_diagnoses = get_top_five(impression_subset.diagnosis)
            for diagnosis, frequency in top5_diagnoses.items():
                #print("      Diagnosis:", diagnosis, frequency)
                top125_paths = top125_paths.append({"complaint":complaint, 
                                                    "impression":impression, 
                                                    "diagnosis":diagnosis, 
                                                    "path_freq":frequency}, ignore_index=True)
    
    # send it out
    return top125_paths

In [8]:
# with all the data:
print("All Paths:")
raw_top_paths = get_top_paths(raw_data)
freq_stats(raw_data.path_freq)
print("top paths:")
freq_stats(raw_top_paths.path_freq)

All Paths:
   count: 40131
   sum: 700499
   mean: 17.45530886347213
   std: 43.69302294514698
   iqr: 8.0
   quantiles:[ 1 ][ 1 ][ 2 ][ 9 ][ 272 ]

top paths:
   count: 125
   sum: 3126
   mean: 25.008
   std: 35.418351357780985
   iqr: 18.0
   quantiles:[ 3 ][ 8 ][ 12 ][ 26 ][ 272 ]



In [9]:
# only paths that occur 5+ times
print("Freq ≥ 5:")
fiveplus_data = raw_data.loc[raw_data.path_freq >= 5]
fiveplus_top_paths = get_top_paths(fiveplus_data)
freq_stats(fiveplus_data.path_freq)
print("top paths:")
freq_stats(fiveplus_top_paths.path_freq)

Freq ≥ 5:
   count: 14575
   sum: 659321
   mean: 45.236432246998284
   std: 63.58616549907544
   iqr: 40.0
   quantiles:[ 5 ][ 8 ][ 16 ][ 48 ][ 272 ]

top paths:
   count: 119
   sum: 3870
   mean: 32.52100840336134
   std: 39.62918132555638
   iqr: 27.0
   quantiles:[ 5 ][ 11 ][ 17 ][ 38 ][ 241 ]



In [10]:
# only paths that occur 10+ times
print("Freq ≥ 10:")
tenplus_data = raw_data.loc[raw_data.path_freq >= 10]
tenplus_top_paths = get_top_paths(tenplus_data)
freq_stats(tenplus_data.path_freq)
print("top paths:")
freq_stats(tenplus_top_paths.path_freq)

Freq ≥ 10:
   count: 9749
   sum: 627261
   mean: 64.34106062160221
   std: 70.2958028241343
   iqr: 74.0
   quantiles:[ 10 ][ 16 ][ 32 ][ 90 ][ 272 ]

top paths:
   count: 87
   sum: 3955
   mean: 45.45977011494253
   std: 52.22131004869901
   iqr: 34.0
   quantiles:[ 10 ][ 14 ][ 26 ][ 48 ][ 272 ]



In [11]:
# honestly can't tell which to go with, so we'll just format them all for testing with the demo
# we're using: raw_top_paths, fiveplus_top_paths, & tenplus_top_paths

# sankey requires that the data be formatted:
#     • as a list of nodes with names & unique ids
#     • and a list of links between 2 nodes and the associated frequencies

# so for each set, we have to:
#     • get a list of unique nodes
#     • split each path into a set of 2 links
#     • replace node names with node ids in the link list

# start by adding "Complaint - ", "Impression - ", & "Diagnosis - " prefixes
def add_prefix(dataframe):
    dataframe["complaint"] = dataframe["complaint"].apply(lambda x: "Complaint - " + x)
    dataframe["impression"] = dataframe["impression"].apply(lambda x: "Impression - " + x)
    dataframe["diagnosis"] = dataframe["diagnosis"].apply(lambda x: "Diagnosis - " + x)
    # don't need a return statement because this modifies the original input in place
    
add_prefix(raw_top_paths)
raw_top_paths.head()
add_prefix(fiveplus_top_paths)
add_prefix(tenplus_top_paths)

In [12]:
# then get list of distinct nodes (returns a dataframe)
def get_nodes(dataframe):
    nodes = pd.DataFrame(np.concatenate((dataframe.complaint.unique(), 
                            dataframe.impression.unique(), 
                            dataframe.diagnosis.unique()), axis=None), columns=["name"])
    nodes["node"] = list(range(0,len(nodes.index)))
    nodes = nodes[["node","name"]]
    return nodes

raw_nodes = get_nodes(raw_top_paths)
print(raw_nodes.head())
fiveplus_nodes = get_nodes(fiveplus_top_paths)
tenplus_nodes = get_nodes(tenplus_top_paths)

   node                                         name
0     0                      Complaint - Sick Person
1     1                            Complaint - Falls
2     2                Complaint - Breathing Problem
3     3  Complaint - Traffic Transportation Incident
4     4          Complaint - Chest Pain NonTraumatic


In [13]:
# separate paths into links
def get_links(dataframe):
    links_ab = pd.DataFrame(data={"source":dataframe.complaint, 
                                  "target":dataframe.impression, 
                                  "value":dataframe.path_freq})
    links_bc = pd.DataFrame(data={"source":dataframe.impression, 
                                  "target":dataframe.diagnosis, 
                                  "value":dataframe.path_freq})
    return pd.concat([links_ab, links_bc], ignore_index=True)

raw_links = get_links(raw_top_paths)
print(raw_links.head())
fiveplus_links = get_links(fiveplus_top_paths)
tenplus_links = get_links(tenplus_top_paths)

                    source                 target value
0  Complaint - Sick Person  Impression - Weakness   114
1  Complaint - Sick Person  Impression - Weakness    33
2  Complaint - Sick Person  Impression - Weakness    32
3  Complaint - Sick Person  Impression - Weakness    26
4  Complaint - Sick Person  Impression - Weakness    24


In [14]:
# encode nodes in link lists using node ids
def encode_links(links, nodes):
    # important to make a new dataframe here so we don't overwrite the original links list
    encoded_links = pd.DataFrame(columns=["source", "target", "value"])
    encoded_links.value = links.value
    encoded_links.source = links.source.apply(lambda x: nodes[nodes.name==x].index[0])
    encoded_links.target = links.target.apply(lambda x: nodes[nodes.name==x].index[0])
    return encoded_links

raw_links_encoded = encode_links(raw_links, raw_nodes)
print(raw_links_encoded.head())
fiveplus_links_encoded = encode_links(fiveplus_links, fiveplus_nodes)
tenplus_links_encoded = encode_links(tenplus_links, tenplus_nodes)

   source  target value
0       0       5   114
1       0       5    33
2       0       5    32
3       0       5    26
4       0       5    24


In [15]:
# combine nodes and links into proper json format
def format_json(nodes, links):
    return '{"nodes":' + nodes.to_json(orient='records') + ',"links":' + links.to_json(orient='records') + '}'

raw_json = format_json(raw_nodes, raw_links_encoded)
fiveplus_json = format_json(fiveplus_nodes, fiveplus_links_encoded)
tenplus_json = format_json(tenplus_nodes, tenplus_links_encoded)

In [16]:
with open("raw.json", "w") as myfile:
    myfile.write(raw_json)

In [17]:
with open("fiveplus.json", "w") as myfile:
    myfile.write(fiveplus_json)

In [18]:
with open("tenplus.json", "w") as myfile:
    myfile.write(tenplus_json)